In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:

def get_daily_wrds(conn,tickers=None):
  
    # Retrieve PERMNOs for the specified tickers
    permnos = conn.get_table(library='crsp', table='stocknames', columns=['permno', 'ticker', 'namedt', 'nameenddt'])
    permnos['nameenddt']=pd.to_datetime(permnos['nameenddt'])
    permnos = permnos[(permnos['ticker'].isin(tickers)) & (permnos['nameenddt']==permnos['nameenddt'].max())]
    # Extract unique PERMNOs
    permno_list = permnos['permno'].unique().tolist()
    print(permno_list)

    # Query daily stock file for the specified PERMNOs
    
    query = f"""
        SELECT permno, date, ret, retx, prc       
        FROM crsp.dsf
        WHERE permno IN ({','.join(map(str, permno_list))})
        ORDER BY date,permno
    """
    daily_returns = conn.raw_sql(query, date_cols=['date'])
    daily_returns = daily_returns.merge(permnos[['permno', 'ticker']], on='permno', how='left')
# Pivot data to have dates as index and tickers as columns
    daily_returns = daily_returns.pivot(index='date', columns='ticker', values='ret').dropna()    
    daily_returns=daily_returns[tickers]

    fama_french = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench",start="1926-01-01")
    daily_data = fama_french[0]  # The first DataFrame contains the daily data
    daily_data.index = pd.to_datetime(daily_data.index)  # Ensure index is datetime
    df_factor = daily_data[['RF','Mkt-RF']]  # Extract the Risk-Free Rate (RF)
    
    daily_returns = daily_returns.merge(df_factor, left_on='Date', right_index=True, how='left')


    print(df_rf.head())  # Display the first few rows



    return daily_returns

# Introduction to Factor models and Estimation issues


Let $r^e$ be an asset excess return over the risk-free rate

Let $f^e$ be a traded factor, for now  the excess returns on SPY ETF

Then we can write

$$r^e=\alpha + b*f+u$$

where $b$ is asset $i$ exposure to the factor $f$

We can always write things that way--it is just a statistical decomposition

1. It can be useful for measuring risk if
    - The risk $u$ is specific to asset $i$ and uncorrelated with other assets
    - I.e the factors picks up lots of the variation in stock returns 

2. It can be useful for measuring expected returns if
    - The premium on the factor f, $E[f]$ summarize well the known "priced" systematic risk factors
    - By "priced" we mean risk-factors that earn a risk premium  $E[f]>0$

We will call the first a factor model of risk, and the second a factor model of expected returns


What use is a factor model?

- the alpha/Beta decomposition is how the money management industry is organized. 
    - Big pay days only if people think you have alpha
    - Getting exposure to the factor is a commodity. Anyone can buy SPY or many other funds that track the market very well
    - In most funds, specially for portfolio managers working a pod shops like Citadel and Millenium, a portfolio manager mandate is to take no factor risk--or at least aim to be close to factor neutral.
    - It is all about the alpha 
- factor models allow you to understand and hedge risk
    - This is essential if you want to concentrate your portfolio only on the pieces that you get compensated


## Alpha and Beta

Note that

$$E[r]=E[rf+r^{e}]=rf+E[\alpha_i + b*f+u]$$

$$E[r]=rf+\alpha+b*E[f]+E[u]$$

- Using that $E[u]=0$ we have
$$E[r]=rf+\alpha+b*E[f]$$

- Thus, just like the risk of our asset could be attributed to the factor and it's component, the same model also allow us to decompose the expected return in terms of the risk-free rate, the premium coming from the common factor exposure $b*E[f]$ and the risk-premium specific to the asset, the ALPHA.


- The alpha does not need to be in a specific asset, can be o a combination of them

- The decomposition works the same for portfolio of assets



## Focus on your edge, hedge all the rest

No matter who you are, you will face dollar risk limits

So it is essential that you deploy risk wisely

The key use of the factor model is to give you discipline to separate your edge from the rest

How to use the factor model?

First , realize that each piece of the factor model will require very thoughtful estimation

Even here with only one asset and one factor we already have to know many things

1. The risk-free rate
2. The beta 
3. The factor risk-premia
4. The factor variance
5. The asset specific alpha
6. The factor specific variance
   
For now we will assume we know each of these.


if you have an asset that has alpha, says, you are confident that APPLE will beat expectations and you expect a 5% appreciation in the next 12 months beyond any factor exposure that APPle might had, what do you do?

1. Buy apple! 

But what is the market crashes? Why bear that risk?  
Portfolio allocators look for managers with alpha, not beta. 
What if the market crashes before apple surprises and your investors pull out?


2. Hedge the market
   - How do you do that?
   - Short it!
   - How much?
   - the asset beta!


Example:

1. You buy 1M in Apple stock, so your portfolio PnL is 1M*$r^{apple}$
2. Your portfolio PnL in excess what you would earn in treasuries is 1M*$(r^{apple}-rf)$
3. Sell (short) 1M*$beta_{apple,f}$ of the factor
4. Now your portfolio is PnL above the risk-free rate is

$$1M*(r^{apple}-rf-\beta*r^{SPY}+\beta*rf)$$

When you sell SPY short you get the money and here I am "investing" it in the risk-free rate

Alternatively you can think of it as using to fund the apple buy

it does not matter, because we are accounting for the time-value of money

We certainly don't want to invest anything else to just add noise to our trade 


$$1M*(r^{apple}-\beta*r^{SPY}-(1-\beta)*rf)$$

For a high beta stock your trade generates extra cash!

If $\beta>1$, $-(1-\beta)*rf>0$

This happens because you are selling more than you are buying

If we substitute our factor model in our trade PnL and take expectations what do we get?

$$1M*E[r^{apple}-rf-\beta*r^{SPY}+\beta*rf)]$$

$$1M*E[\alpha_i+rf+beta*(r^{SPY}-rf)+u_i-rf-\beta*r^{SPY}+\beta*rf)]$$

A pure play portfolio!

$$1M*E[\alpha_i]$$

And what is the portfolio risk?

$$std(1M*(r^{apple}-rf-\beta*r^{SPY}+\beta*rf))=1M*std(u_i)$$





Lets do this!

I will use a regression to get some numbers for us to do this strategy

but for now, just think those as numbers I am giving you and don't think of the regression as giving the right numbers necessarily.


Do example with an asset. Compare the hedged and unhedged returns

Discuss why these are trading strategy returns

Show the positions that Implement this trade


Look the advantadges of minimize risk



What is the probability that your portfolio will be worth less that 750K in the end of the year if you don't hedge?

What is the probability of this same loss if you hedge?

Suppose you have access to funding at rf, how much larger a position can you take with the hedge while keeping the same probability of loss of 250K?

## How do we know these things?


  






## Estimation of a factor model